<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка</a></span><ul class="toc-item"><li><span><a href="#Удаление-столбца-Unnamed:-0" data-toc-modified-id="Удаление-столбца-Unnamed:-0-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Удаление столбца Unnamed: 0</a></span></li><li><span><a href="#Лемматизация,-приведение-текста-к-нижнему-регистру-и-очистка-текста" data-toc-modified-id="Лемматизация,-приведение-текста-к-нижнему-регистру-и-очистка-текста-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Лемматизация, приведение текста к нижнему регистру и очистка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Подготовка-признаков-и-разделение-данных-на-выборки" data-toc-modified-id="Подготовка-признаков-и-разделение-данных-на-выборки-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Подготовка признаков и разделение данных на выборки</a></span></li><li><span><a href="#Проверка-на-пропуски" data-toc-modified-id="Проверка-на-пропуски-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Проверка на пропуски</a></span></li><li><span><a href="#Тестирование-моделей" data-toc-modified-id="Тестирование-моделей-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Тестирование моделей</a></span></li><li><span><a href="#Тестирование-на-тестовой-выборке" data-toc-modified-id="Тестирование-на-тестовой-выборке-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Тестирование на тестовой выборке</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Обучение модели классификации комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Требуется обучить модель классифицировать комментарии на позитивные и негативные.

Условия заказчика: построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
from IPython.display import display, HTML
HTML("<style>.container { width:90% !important; }</style>")

## Подготовка

In [2]:
import os
import pandas as pd
import numpy as np
import nltk
import re
import spacy
import pickle

from tqdm import tqdm
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [3]:
np.random.seed(12345) # Фиксация псевдослучайности

In [4]:
pth = 'toxic_comments.csv'

if os.path.exists(pth):
    df = pd.read_csv(pth)
else:
    cloud_url = ''
    try:
        df = pd.read_csv(cloud_url)
    except:
        print("Failed to load data from the cloud.")

In [5]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


**Вывод:** Из первого взгляда на данные можно сказать
- Нужно привести текст к нижнему регистру, то поможет снизить размерность данных и сделает слова с разным регистром однородными
- Требуется удалить лишние символы: можно удалить пунктуацию, специальные символы и цифры, так как они могут не нести смысловую нагрузку
- Нужно удалить столбец Unnamed: 0, так как он просто копирует индексы и не несет информационной нагрузки

## Предобработка

### Удаление столбца Unnamed: 0

In [7]:
df = df.drop(columns='Unnamed: 0')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Лемматизация, приведение текста к нижнему регистру и очистка текста

In [8]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize(texts):
    print("1. Входные тексты:", texts[:2])
    
    # Приведение текстов к нижнему регистру
    texts_lower = [text.lower() for text in texts]
    print("2. Тексты в нижнем регистре:", texts_lower[:2])
    
    # Удаление всех символов, кроме английских букв и пробелов, с помощью регулярных выражений
    texts_cleaned = [re.sub(r'[^a-z\']', ' ', text) for text in texts_lower]
    print("3. Очищенные тексты:", texts_cleaned[:2])
    
    # Разделение слов пробелами, чтобы получить чистые слова
    texts_cleaned_split = [" ".join(text.split()) for text in texts_cleaned]
    print("4. Разделенные слова:", texts_cleaned_split[:2])
    
    # Лемматизация текстов с помощью Spacy в пакетном режиме
    docs = list(nlp.pipe(texts_cleaned_split, batch_size=1000))
    lemmatized_texts = [' '.join([token.lemma_ for token in doc]) for doc in docs]
    print("5. Лемматизированные тексты:", lemmatized_texts[:2])
    
    return lemmatized_texts

In [9]:
# Попробуем загрузить данные из файла, если файл существует
try:
    with open("lemmatized_texts.pkl", "rb") as file:
        corpus = pickle.load(file)
    print("Данные успешно загружены из файла.")
    
except FileNotFoundError:
    # Если файл не найден, выполняем лемматизацию
    corpus = df['text'].values
    corpus = lemmatize(corpus)
    
    # Записываем результаты лемматизации в файл
    with open("lemmatized_texts.pkl", "wb") as file:
        pickle.dump(corpus, file)
    print("Результаты лемматизации сохранены в файл.")

print('Очищенный и лемматизированный текст:', corpus[0])

Данные успешно загружены из файла.
Очищенный и лемматизированный текст: explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york dolls fac and please do not remove the template from the talk page since I be retire now


**Вывод:** в ходе предобработки данных
- Удален столбец Unnamed
- Проведена лемматизация, а также приведение к нижнему регистру и очистка текста

## Обучение

### Подготовка признаков и разделение данных на выборки

In [10]:
# Преобразовываем целевой признак в массив
y = df['toxic'].values

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(corpus, y, stratify=y)

train_percentage = len(X_train) / len(corpus) * 100
test_percentage = len(X_test) / len(corpus) * 100

print(f"Train data percentage: {train_percentage}%")
print(f"Test data percentage: {test_percentage}%")

Train data percentage: 75.0%
Test data percentage: 25.0%


In [11]:
nltk.download('stopwords')
stopwords = nltk_stopwords.words('english')
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yarma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Проверка на пропуски

In [12]:
# Проверяем наличие пропусков в X_train_tfidf
has_nan_train = np.isnan(X_train_tfidf.data).any()
print(f"Пропуски в X_train_tfidf: {has_nan_train}")

# Проверяем наличие пропусков в X_test_tfidf
has_nan_test = np.isnan(X_test_tfidf.data).any()
print(f"Пропуски в X_test_tfidf: {has_nan_test}")

# Проверяем наличие нулевых элементов в X_train_tfidf
has_zeros_train = (X_train_tfidf.data == 0).any()
print(f"Нули в X_train_tfidf: {has_zeros_train}")

# Проверяем наличие нулевых элементов в X_test_tfidf
has_zeros_test = (X_test_tfidf.data == 0).any()
print(f"Нули в X_test_tfidf: {has_zeros_test}")

Пропуски в X_train_tfidf: False
Пропуски в X_test_tfidf: False
Нули в X_train_tfidf: False
Нули в X_test_tfidf: False


In [13]:
# Привести признаки TF-IDF к типу np.float32 для экономии памяти 
X_train_tfidf = X_train_tfidf.astype(np.float32)
X_test_tfidf = X_test_tfidf.astype(np.float32)

### Тестирование моделей 

In [14]:
# Определим различные модели для классификации
models = {
    'CatBoostClassifier': CatBoostClassifier(),
    'LGBMClassifier': LGBMClassifier(),
    'LogisticRegression': LogisticRegression()
}

# Настроим сетки гиперпараметров для каждой модели
param_grid_catboost = {
    'learning_rate': [0.1],
    'depth': [6],
    'verbose': [250],
    'eval_metric': ['F1']
}

param_grid_lightgbm = {
    'learning_rate': [0.1],
    'max_depth': [6],
    'num_leaves': [31],
    'objective': ['binary'],
    'metric': ['binary_logloss']
}

param_grid_logistic_regression = {
    'C': [1, 10, 100],
    'max_iter' : [1000]
}

param_grids = {
    'CatBoostClassifier': param_grid_catboost,
    'LGBMClassifier': param_grid_lightgbm,
    'LogisticRegression': param_grid_logistic_regression
}

# Используем стратифицированную кросс-валидацию
cv = StratifiedKFold(n_splits=3, shuffle=True)

best_params_dict = {}

for model_name, model in models.items():
    param_grid_model = param_grids[model_name]
    
    grid_search = GridSearchCV(model, param_grid_model, cv=cv, scoring='f1', error_score='raise', n_jobs=-1)
    grid_search.fit(X_train_tfidf, y_train)
    
    best_params_dict[model_name] = grid_search.best_params_

    print(f"Лучшие параметры для модели {model_name}:")
    print(grid_search.best_params_)
    
    best_f1 = grid_search.best_score_
    
    print(f"Лучший F1-скор для модели {model_name}:")
    print(best_f1)
    
    if best_f1 > 0.75:  
        print("F1-скор соответствует требованиям. Качество модели хорошее.")
    else:
        print("F1-скор требует улучшения для данной модели.")
    
    print()

0:	learn: 0.4962580	total: 569ms	remaining: 9m 28s
250:	learn: 0.7368579	total: 1m 56s	remaining: 5m 47s
500:	learn: 0.7778691	total: 3m 53s	remaining: 3m 52s
750:	learn: 0.7968728	total: 5m 45s	remaining: 1m 54s
999:	learn: 0.8124352	total: 7m 33s	remaining: 0us
Лучшие параметры для модели CatBoostClassifier:
{'depth': 6, 'eval_metric': 'F1', 'learning_rate': 0.1, 'verbose': 250}
Лучший F1-скор для модели CatBoostClassifier:
0.7532364592726895
F1-скор соответствует требованиям. Качество модели хорошее.

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 12140, number of negative: 107329
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.687420 seconds.
You can set `force_row_wise=true` to remove the overhe

**Вывод:** в ходе данного этапа проекта
- Подготовлены признаки 
- Данные разделены на выборки 
- Проведена проверка на пропуски и нулевые значения в тренировочной и тестовой выборках
- В ходе обучения моделей лучший результат показала LogisticRegression (F1~0.76), протестируем ее на тестовой выборке

### Тестирование на тестовой выборке 

In [15]:
model = LogisticRegression(**best_params_dict['LogisticRegression'])

model.fit(X_train_tfidf, y_train)

predict = model.predict(X_test_tfidf)

print(f1_score(y_test, predict))

0.7787099436116077


**Комментарий:**
F1 LogisticRegression на тестовой выборке составила ~0.78, что соответствует требованиям закачника

## Общий вывод

Проект "Классификация комментариев в интернет-магазине" был успешно выполнен с целью построения модели, способной классифицировать комментарии на позитивные и негативные. Этот инструмент необходим для нового сервиса интернет-магазина "Викишоп", который позволяет пользователям редактировать описания товаров, как в вики-сообществах.

В ходе проекта были выполнены следующие ключевые шаги:

    Предобработка данных: текст комментариев был приведен к нижнему регистру, произведена лемматизация, а также удалены лишние символы, такие как пунктуация, специальные символы и цифры. Эти шаги помогли снизить размерность данных и сделали текст более однородным.

    Подготовка признаков: тексты комментариев были преобразованы в числовые векторы с использованием метода TF-IDF.

    Обучение моделей: были опробованы различные алгоритмы машинного обучения, такие как CatBoost, LightGBM и Logistic Regression. Лучший результат показала модель Logistic Regression с F1-скором около 0.76.

    Оценка результатов: выбранная модель была протестирована на тестовой выборке, и F1-скор составил около 0.78, что соответствует требованиям заказчика.

    Заключение: Проект успешно достиг цели построения модели для классификации комментариев. Разработанный инструмент поможет интернет-магазину "Викишоп" автоматизировать процесс модерации комментариев, обеспечивая безопасное и комфортное взаимодействие пользователей с описаниями товаров.

В целом, успешное завершение проекта позволит интернет-магазину "Викишоп" обеспечить более безопасное и качественное взаимодействие пользователей с комментариями, повысив уровень доверия и удовлетворенности клиентов.